In [1]:
from bs4 import BeautifulSoup
import requests,time
import pandas as pd



In [29]:
base_url = "https://www.stproperty.sg"
url = "https://www.stproperty.sg/singapore-property-listings/hdb-for-sale"
result = requests.get(url)
doc = BeautifulSoup(result.text, "html.parser")

In [11]:
last_page = int(doc.find_all(class_ = "hidden-xs")[-1].text.strip())

In [3]:
def find_all_hdb_in_page(doc):
    tags = doc.find_all(class_ = "listingContainer")
    detailed_urls = [];
    for tag in tags:
        url = tag.find(class_ = "listingDetailsDivLink")
        url = url.attrs['href']
        detailed_urls.append(base_url + url)
    return detailed_urls


In [4]:
def format_price(price):
    price = price.text.strip()
    price = price.replace(",", "")
    return int(price)

In [44]:
def get_description(url): 
    result = requests.get(url)
    doc = BeautifulSoup(result.text, "html.parser")
    try:
        town = doc.find(text = "HDB Town")
        town = town.parent.find_next_sibling().text
    except Exception:
        town = None;
    price = doc.find(class_ = "listing-price")
    price =  format_price(price.find('span', {"itemprop":"price"}))
    size = doc.find(text = "Size")
    size = size.parent.find_next_sibling().find("span", {"itemprop":"value"}).text
    size = int(size.split()[0].replace(",",""))
    try: 
        bedrooms = doc.find(text= 'Bedrooms')
        bedrooms = int(bedrooms.parent.find_next_sibling().text)
    except ValueError:
        bedrooms = 0
        print(town)
    except AttributeError:
        bedrooms = 0
    # if no bathrooms on website, assign bathrooms to be one
    try:
        bathrooms = doc.find(text = "Bathrooms")
        bathrooms = int(bathrooms.parent.find_next_sibling().text)
    except AttributeError:
        bathrooms = 0
    except ValueError:
        bathrooms = 0
    details = {"town" : town, "price": price, "size": size , "bedrooms":bedrooms, 'bathrooms':bathrooms}
    return details

In [ ]:
data = []

In [54]:
i = 1;
for i in range(101, 121):
    next_page_url = f"/?page={i}"
    updated_url = url + next_page_url
    print(updated_url)
    result = requests.get(updated_url);
    doc = BeautifulSoup(result.text, "html.parser")
    detailed_url = find_all_hdb_in_page(doc)
    print(len(detailed_url))
    for u in detailed_url:
        time.sleep(1)
        data.append(get_description(u))
    



https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=101
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=102
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=103
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=104
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=105
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=106
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=107
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=108
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=109
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=110
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=111
20
https://www.stproperty.sg/singapore-property-listings/hdb-for-sale/?page=112
20
https://www.stproperty.sg/singapore-prop

In [55]:
len(data)
# data

2235

In [57]:
df = pd.DataFrame(data)

In [61]:
df.to_csv("hdb_details.csv", index=False)

In [60]:
df

,town,price,size,bedrooms,bathrooms
0,Woodlands,488000,101,3,2
1,Queenstown,468000,65,2,2
2,Serangoon,400000,70,2,2
3,Yishun,670000,112,3,2
4,Hougang,450000,67,2,2
...,...,...,...,...,...
2230,Punggol,368000,37,1,1
2231,Tampines,468000,74,2,2
2232,Tampines,490000,84,3,2
2233,Punggol,728000,110,3,2
